In [11]:
## import pytorch and related packages

import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import transforms
from torchvision.models import vgg16, VGG16_Weights

In [12]:
## import other packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import PIL
import glob
import os
from IPython.display import Image

In [13]:
## setup device for training 

if torch.backends.mps.is_available():    # check if Apple Silicon mps is available
    device = "mps"
elif torch.cuda.is_available():          # check id cuda is available
    device = "cuda"
else:
    device = "cpu"                       # default to cpu if none are available

device

'mps'

In [14]:
## Image processing 

transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

In [15]:
## Prepating the data 

train_path = "./Data/train/"
val_path = "./Data/valid/"

train_dataset = datasets.ImageFolder(train_path, transform=transform)
val_dataset = datasets.ImageFolder(val_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [16]:
## Declaring the model

model = vgg16(weights=VGG16_Weights.DEFAULT)

for param in model.features.parameters():
    param.requires_grad = False

print((model.classifier))
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Flatten(),
    nn.BatchNorm1d(num_features=25088), 
    nn.Linear(25088, 32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(32, 32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(32, 32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Linear(32, 4),
    nn.Softmax(dim=1)
)

# for param in model.fc.parameters():
#     param.requires_grad = True

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [17]:
## Set up loss function and optimizer

loss_fn = nn.CrossEntropyLoss().to(device=device)

optimizer = torch.optim.Adam(model.parameters())

lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.50, patience=3, verbose=True, min_lr=1e-7)

# optimizer = torch.optim.Adam(params=model.fc.parameters(),


# optimizer = torch.optim.AdamW(params=model.fc.parameters(),
#                              lr=0.001,
#                              weight_decay=0.0001)

In [18]:
## Training loop

epochs = 30
model = model.to(device=device)

for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0

    model.train()

    for (X, y) in train_loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(X)
        loss = loss_fn(logits, y)
        _, preds = torch.max(logits, 1)
        train_acc += torch.sum(preds == y).item()

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * y.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset)


    val_loss = 0.0    
    val_acc = 0
    
    model.eval()
    with torch.no_grad():
        for (X, y) in val_loader:
            X, y = X.to(device), y.to(device)

            logits = model(X)
            loss = loss_fn(logits, y)

            _, preds = torch.max(logits, 1)
            val_acc += torch.sum(preds == y).item()

            val_loss += loss.item() * y.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_acc /len(val_loader.dataset)

    lr_scheduler.step(val_loss)
    
    print(f'Epoch {epoch + 1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | 'f'Test Loss: {val_loss:.4f} | Test Acc: {val_acc:.4f}')



Epoch 1/30 | Train Loss: 1.3654 | Train Acc: 0.3475 | Test Loss: 1.3348 | Test Acc: 0.5556
Epoch 2/30 | Train Loss: 1.3169 | Train Acc: 0.4715 | Test Loss: 1.2768 | Test Acc: 0.6528
Epoch 3/30 | Train Loss: 1.2576 | Train Acc: 0.5742 | Test Loss: 1.2297 | Test Acc: 0.6250
Epoch 4/30 | Train Loss: 1.2109 | Train Acc: 0.6117 | Test Loss: 1.1796 | Test Acc: 0.6667
Epoch 5/30 | Train Loss: 1.1588 | Train Acc: 0.6819 | Test Loss: 1.1245 | Test Acc: 0.7083
Epoch 6/30 | Train Loss: 1.0983 | Train Acc: 0.7569 | Test Loss: 1.0847 | Test Acc: 0.7917
Epoch 7/30 | Train Loss: 1.0484 | Train Acc: 0.8369 | Test Loss: 1.0217 | Test Acc: 0.8611
Epoch 8/30 | Train Loss: 1.0311 | Train Acc: 0.8320 | Test Loss: 0.9909 | Test Acc: 0.8750
Epoch 9/30 | Train Loss: 0.9650 | Train Acc: 0.8809 | Test Loss: 0.9501 | Test Acc: 0.9167
Epoch 10/30 | Train Loss: 0.9507 | Train Acc: 0.8842 | Test Loss: 0.9305 | Test Acc: 0.8611
Epoch 11/30 | Train Loss: 0.9243 | Train Acc: 0.8874 | Test Loss: 0.9249 | Test Acc: 0.87